# GERENCIAL ALFSTS/DIREP/EQPEM

In [1]:
from IPython.display import HTML
import gerencial_exp as g

2022-02-18 11:27:44,602 ajna         INFO     Configuração de log efetuada


Fazendo log de erros e alertas no arquivo  ../..\..\ajna_docs\commons\ajna_commons\flask\error.log
Fazendo log de atividade no arquivo  ../..\..\ajna_docs\commons\ajna_commons\flask\access.log


# Verificações por inspeção não invasiva

In [2]:
import os
import sys
from datetime import datetime

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

WIDTH = 1100
pd.options.display.float_format = 'R$ {:,.2f}'.format

caminho_commons = os.path.join('../..', '..', 'ajna_docs', 'commons')
caminho_virasana = os.path.join('../..', '..', 'ajna_docs', 'virasana')
sys.path.append(caminho_commons)
sys.path.append('../..')
sys.path.append(caminho_virasana)

from bhadrasana.models import engine

In [3]:
SQL = '''SELECT c.numeroCEmercante, c.descricao, c.embarcador, c.portoDestFinal, c.consignatario,
 c.portoOrigemCarga, n.codigoConteiner, n.identificacaoNCM
FROM conhecimentosresumo c INNER JOIN ncmitemresumo n ON c.numeroCEmercante = n.numeroCEMercante
WHERE c.create_date <= "2022-02-14" AND c.create_date >=  "2022-02-13"
 AND c.portoDestFinal = "BRSSZ" 
 ORDER BY c.numeroCEmercante
 LIMIT 100
 '''

df = pd.read_sql(SQL, engine)